# Analyse de la dist. du vocabulaire

### Imports

In [4]:
import nltk
nltk.download('stopwords') #liste pré-définie de stopwords
from nltk.corpus import stopwords #attention de bien nettoyer les stopwords !! sinon résultats bizarres

import os
import yake #(Yet Another Keyword Extractor) "that uses text statistical features to select the most important keywords from a document"

from collections import Counter
from wordcloud import WordCloud #besoin de compilateur c++ pour l'installer

from IPython.display import Image

import re #pour les expressions régulières afin de nettoyer l OCRisation

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\casia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Gerer les stopwords

#### Créer une liste des stopwords

In [2]:
#jouer encore avec cela pour en ajouter des nouveaux en fonction du contexte, modifié de la liste de https://github.com/stopwords-iso/stopwords-fr?tab=readme-ov-file, à verifer sa fidelité
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout", "elles",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", ":", "même", "voir", "fois", "jour", "année", "ans", "faites", "le", "la", "de"]

# New stopwords to add
git_stopwords = [
    "a", "abord", "absolument", "afin", "ah", "ai", "aie", "aient", "aies", "ailleurs", "ainsi", "ait", "allaient", "allo", "allons", "allô", "alors", "anterieur", "anterieure", "anterieures", "apres", "après", "as", "assez", "attendu", "au", "aucun", "aucune", "aucuns", "aujourd", "aujourd'hui", "aupres", "auquel", "aura", "aurai", "auraient", "aurais", "aurait", "auras", "aurez", "auriez", "aurions", "aurons", "auront", "aussi", "autant", "autre", "autrefois", "autrement", "autres", "autrui", "aux", "auxquelles", "auxquels", "avaient", "avais", "avait", "avant", "avec", "avez", "aviez", "avions", "avoir", "avons", "ayant", "ayez", "ayons", "bah", "bas", "basee", "bat", "beau", "beaucoup", "bien", "bigre", "bon", "boum", "bravo", "car", "ce", "ceci", "cela", "celle", "celle-ci", "celle-là", "celles", "celles-ci", "celles-là", "celui", "celui-ci", "celui-là", "celà", "cent", "cependant", "certain", "certaine", "certaines", "certains", "certes", "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "chacun", "chacune", "chaque", "cher", "chers", "chez", "chiche", "chut", "chère", "chères", "ci", "cinq", "cinquantaine", "cinquante", "cinquantième", "cinquième", "clac", "clic", "combien", "comme", "comment", "comparable", "comparables", "compris", "concernant", "contre", "couic", "crac", "dans", "de", "debout", "dedans", "dehors", "deja", "delà", "depuis", "dernier", "derniere", "derriere", "derrière", "des", "desormais", "desquelles", "desquels", "dessous", "dessus", "deux", "deuxième", "deuxièmement", "devant", "devers", "devra", "devrait", "different", "differentes", "differents", "différent", "différente", "différentes", "différents", "dire", "directe", "directement", "dit", "dite", "dits", "divers", "diverse", "diverses", "dix", "dix-huit", "dix-neuf", "dix-sept", "dixième", "doit", "doivent", "donc", "dont", "dos", "douze", "douzième", "dring", "droite", "du", "duquel", "durant", "dès", "début", "désormais", "effet", "egale", "egalement", "egales", "eh", "elle", "elle-même", "elles", "elles-mêmes", "en", "encore", "enfin", "entre", "envers", "environ", "es", "essai", "est", "et", "etant", "etc", "etre", "eu", "eue", "eues", "euh", "eurent", "eus", "eusse", "eussent", "eusses", "eussiez", "eussions", "eut", "eux", "eux-mêmes", "exactement", "excepté", "extenso", "exterieur", "eûmes", "eût", "eûtes", "f", "fais", "faisaient", "faisant", "fait", "faites", "façon", "feront", "fi", "flac", "floc", "fois", "font", "force", "furent", "fus", "fusse", "fussent", "fusses", "fussiez", "fussions", "fut", "fûmes", "fût", "fûtes", "gens", "haut", "hein", "hem", "hep", "holà", "hop", "hormis", "hors", "hou", "houp", "hue", "hui", "huit", "huitième", "hum", "hurrah", "hé", "hélas", "i", "ici", "il", "ils", "importe", "j", "je", "jusqu", "jusque", "juste", "la", "laisser", "laquelle", "las", "le", "lequel", "les", "lesquelles", "lesquels", "leur", "leurs", "longtemps", "lors", "lorsque", "lui", "lui-meme", "lui-même", "là", "lès", "ma", "maint", "maintenant", "mais", "malgre", "malgré", "maximale", "me", "meme", "memes", "merci", "mes", "mien", "mienne", "miennes", "miens", "mille", "mince", "mine", "minimale", "moi", "moi-meme", "moi-même", "moindres", "moins", "mon", "mot", "moyennant", "multiple", "multiples", "même", "mêmes", "na", "n'a" "naturel", "naturelle", "naturelles", "ne", "neanmoins", "necessaire", "necessairement", "neuf", "neuvième", "ni", "nombreuses", "nombreux", "nommés", "non", "nos", "notamment", "notre", "nous", "nous-mêmes", "nouveau", "nouveaux", "nul", "néanmoins", "nôtre", "nôtres", "oh", "ohé", "ollé", "olé", "on", "ont", "onze", "onzième", "ore", "ou", "ouf", "ouias", "oust", "ouste", "outre", "ouvert", "ouverte", "ouverts", "où", "paf", "par", "parce", "parfois", "parle", "parlent", "parler", "parmi", "parole", "parseme", "partant", "particulier", "particulière", "particulièrement", "pas", "passé", "pendant", "pense", "permet", "personne", "personnes", "peu", "peut", "peuvent", "peux", "pff", "pfft", "pfut", "pif", "pire", "pièce", "plein", "plouf", "plupart", "plus", "plusieurs", "plutôt", "possessif", "possessifs", "possible", "possibles", "pouah", "pour", "pourquoi", "pourrais", "pourrait", "pouvait", "prealable", "precisement", "premier", "première", "premièrement", "pres", "probable", "probante", "procedant", "proche", "près", "psitt", "pu", "puis", "puisque", "pur", "pure", "qu", "quand", "quant", "quant-à-soi", "quanta", "quarante", "quatorze", "quatre", "quatre-vingt", "quatrième", "quatrièmement", "que", "quel", "quelconque", "quelle", "quelles", "quelqu'un", "quelque", "quelques", "quels", "qui", "quiconque", "quinze", "quoi", "quoique", "r", "rare", "rarement", "rares", "relative", "relativement", "remarquable", "rend", "rendre", "restant", "reste", "restent", "restrictif", "retour", "revoici", "revoilà", "rien", "sa", "sacrebleu", "sait", "sans", "sapristi", "sauf", "se", "sein", "seize", "selon", "semblable", "semblaient", "semble", "semblent", "sent", "sept", "septième", "sera", "serai", "seraient", "serais", "serait", "seras", "serez", "seriez", "serions", "serons", "seront", "ses", "seul", "seule", "seulement", "si", "sien", "sienne", "siennes", "siens", "sinon", "six", "sixième", "soi", "soi-même", "soient", "sois", "soit", "soixante", "sommes", "son", "sont", "sous", "souvent", "soyez", "soyons", "specifique", "specifiques", "speculatif", "stop", "strictement", "subtiles", "suffisant", "suffisante", "suffit", "suis", "suit", "suivant", "suivante", "suivantes", "suivants", "suivre", "sujet", "superpose", "sur", "surtout", "ta", "tac", "tandis", "tant", "tardive", "te", "tel", "telle", "tellement", "telles", "tels", "tenant", "tend", "tenir", "tente", "tes", "tic", "tien", "tienne", "tiennes", "tiens", "toc", "toi", "toi-même", "ton", "touchant", "toujours", "tous", "tout", "toute", "toutefois", "toutes", "treize", "trente", "tres", "trois", "troisième", "troisièmement", "trop", "très", "tsoin", "tsouin", "tu", "té", "un", "une", "unes", "uniformement", "unique", "uniques", "uns", "va", "vais", "valeur", "vas", "vers", "via", "vif", "vifs", "vingt", "vivat", "vive", "vives", "vlan", "voici", "voie", "voient", "voilà", "voire", "vont", "vos", "votre", "vous", "vous-mêmes", "vu", "vé", "vôtre", "vôtres", "zut", "à", "â", "ça", "ès", "étaient", "étais", "était", "étant", "état", "étiez", "étions", "été", "étée", "étées", "étés", "êtes", "être", "ô"
]

#domaine specific 
specific_stopwords = [
"rue", "-", "drapeau", "DRAPEAU", "LE", "ROUGE", "com", "qu'il", "d'une", "d'un", "l'on", "qu'il", "qu'ils", "qu'elle", "qu'elles"
#garder les noms des villes pour faire une analyse spatiale? 
#does it also include common first names?
#does it also include uppercase versions?
#à voir : soir, matin = seront pe liés au nom du journal et non du temps de la journée, define what a stopword means in this context
]

sw += git_stopwords
sw += specific_stopwords
sw = set(sw)


print(f"{len(sw)} stopwords:\n {sorted(sw)}")

700 stopwords:
 ['-', ':', 'DRAPEAU', 'LE', 'ROUGE', 'a', 'abord', 'absolument', 'afin', 'ah', 'ai', 'aie', 'aient', 'aies', 'ailleurs', 'ainsi', 'ait', 'allaient', 'allo', 'allons', 'allô', 'alors', 'année', 'ans', 'anterieur', 'anterieure', 'anterieures', 'apres', 'après', 'as', 'assez', 'attendu', 'au', 'aucun', 'aucune', 'aucuns', 'aujourd', "aujourd'hui", 'aupres', 'auquel', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autant', 'autre', 'autrefois', 'autrement', 'autres', 'autrui', 'aux', 'auxquelles', 'auxquels', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', 'avons', 'ayant', 'ayante', 'ayantes', 'ayants', 'ayez', 'ayons', 'bah', 'bas', 'basee', 'bat', 'beau', 'beaucoup', 'bien', 'bigre', 'bon', 'boum', 'bravo', 'c', 'car', 'ce', 'ceci', 'cela', 'celle', 'celle-ci', 'celle-là', 'celles', 'celles-ci', 'celles-là', 'celui', 'celui-ci', 'celui-là', 'celà', 'cent', 'cependant'

#### Tokeniser

Créer un fichier bash qui concatène tous les fichiers txt de halley

In [3]:
#merci les profs
path = "../../data/halley/halley_txt/" # Path to the directory containing text files

with open("../../data/halley/halley_all.txt", "w", encoding="utf-8") as output_file:
    for file in os.listdir(path):
        if file.endswith(".txt"):
            with open(os.path.join(path, file), "r", encoding="utf-8") as f:
                output_file.write(f.read())

# Récupération du contenu du fichier bash
path = "../../data/halley/halley_all.txt"
limit = 10**8

with open(path, 'r', encoding='utf-8') as f:
    text = f.read()[:limit]

# Tokenization, sw pas encore appliqués
words = nltk.wordpunct_tokenize(text) #tokenized words from the concatenated Halley corpus (halley_all.txt)
print(f"{len(words)} words found")
words[:50] 

# a big list of all of the keywords from all of the files, and then do a frequency analysis on that list to see what are the most common keywords across all of the texts = kinda loses all temporal analysis but ok for the overarching ALL keywords, 
# we should expect only the most common elements to come out? but key words isn't most common... careful

4858920 words found


['yF',
 ':',
 'LE',
 'DRAPEAU',
 'ROUGE',
 'Edison',
 'revendique',
 'la',
 'priorité',
 'de',
 'l',
 "'",
 'invention',
 'du',
 'phonographe',
 '.-',
 'On',
 'a',
 'signalé',
 'dernièrement',
 'la',
 'mort',
 'du',
 'poète',
 'ot',
 "'",
 'savant',
 'français',
 'Charles',
 'Croa',
 'considéré',
 'com',
 '-',
 ',',
 'mo',
 'l',
 "'",
 'inventeur',
 'du',
 'phonographe',
 ',',
 'dont',
 'il',
 'aurait',
 'décrit',
 ',',
 'le',
 'premier',
 ',',
 'le']

#### Eliminer les stopwords et les termes non alphabétiques

In [ ]:
#shows words after filtering stop words, much better
kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(kept) #creates a unique vocabulary (no duplicates)
print(f"{len(kept)} words kept ({len(voc)} different word forms)")
kept[:20] 
#1587064 words kept (220119 different word forms) 
#seems like a small difference but it's actually around 1500 words removed by the new stopword list


#these still aren't the words I'd expect... double check the source
#AH it's because the WHOLE page of ALL the pages isn't about halleys comet 
# do we go for an analysis of co-text as well then? 

1587064 words kept (220119 different word forms)


['rouge',
 'edison',
 'revendique',
 'priorité',
 'invention',
 'phonographe',
 'signalé',
 'dernièrement',
 'mort',
 'poète',
 'savant',
 'français',
 'charles',
 'croa',
 'considéré',
 'inventeur',
 'phonographe',
 'décrit',
 'mécanisme',
 'edison']

In [12]:
kept = [w.lower() for w in words 
        if len(w) > 2 
        and w.isalpha() 
        and w.lower() not in sw
        and not re.search(r'(.)\1{2,}', w.lower())]

voc = set(kept) #creates a unique vocabulary (no duplicates)
print(f"{len(kept)} words kept ({len(voc)} different word forms)")
kept[:20] 
#avec 5 caractères le même, 1586939 words kept (220004 different word forms) quand même une différence mais elle est assez minimale, dc environs 100 mots en moins 
#avec 3 carartères, 1586631 words kept (219739 different word forms)
#avec 2, 1584198 words kept (217992 different word forms)

1584198 words kept (217992 different word forms)


['rouge',
 'edison',
 'revendique',
 'priorité',
 'invention',
 'phonographe',
 'signalé',
 'dernièrement',
 'mort',
 'poète',
 'savant',
 'français',
 'charles',
 'croa',
 'considéré',
 'inventeur',
 'phonographe',
 'décrit',
 'mécanisme',
 'edison']

#### Calculer la taille du vocabulaire

In [8]:
#récupérer la fréquence des mots sur toute le dataset 
fdist = nltk.FreqDist(kept) #yes, kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
fdist.most_common(10)

#heures might be on the chopping block for sw 

#voisins plus proches to halley would be useful
#or maybe we do a co-text analysis of words around halley?

[('heures', 4175),
 ('bruxelles', 3613),
 ('mai', 3111),
 ('grand', 3095),
 ('prix', 2688),
 ('temps', 2602),
 ('lieu', 2568),
 ('comète', 2556),
 ('francs', 2549),
 ('soir', 2445)]

#### Variation from structure of modules 
I don't see what the plot adds, so I'm leaving it out for now
I'm also leaving out les mots qui n'apparaissent qu'une fois dans le corpus et les plus longs pour l'instant

In [17]:
#les mots les plus longs
n = 150
sorted(voc, key=len, reverse=True)[:n]

#oh TERRIBLE OCR without the regex, not a single word, but a couple of conjuctions at the end 

#quand meme une grosse différence avec le regex pour enlever les mots avec des lettres répétées
#peut être qu'on peut faire un truc plus fin pour garder les mots légitimes, c'est un peu dommage que c'est mots ne seront pas analysés dans l'état quelles sont mtn

['wasanmmaâeaanssnnaéémmqwakmnscâéahaai',
 'siiïlgipioïinciaiflgstibïagipiibïfgs',
 'laterreetsemontrependantquelques',
 'vbienterprogressîtomentlesdoses',
 'nanisationdestribusalbanaises',
 'ffanceestencoursdenégociation',
 'reconnaissanceestenflnrendu',
 'liqiicsdonnentrégulièrement',
 'lechiffredesfréqùentations',
 'disccmnsdecircemstajieefut',
 'hautenbasaupetitconducleur',
 'défenderessesjprétendaient',
 'agencemaritimedekeyserthnm',
 'respeesèeartueuesenjentle',
 'celaetplusquejamaiscesont',
 'industrielsetgommerçints',
 'faulcroirelesliistoriens',
 'curieuxquionteupourobjet',
 'trouyalèntilarjueùtenant',
 'naireouplutôtsavonpurol',
 'pourlasociétéanonymeaes',
 'correapondantparuculier',
 'milièuxfoitternemsntaux',
 'vaccinationantityphique',
 'quainfientviesipeintres',
 'femmecatherinètuytters',
 'adtaiitenemfoqteifyehi',
 'intéressantevloutefois',
 'conduisentvdirectement',
 'àlaoïnectlonoujcullnal',
 'mentsnouveauxcapablesd',
 'gouverneinéïïfiivavoir',
 'approxlniiaitiivamieat

### Keywords Analysis 

#### Extraire les keywords du texte sans stopwords

Statistical approach: the lower the score, the more relevant the keyword is.
import yake

text = "xxx "

Simple usage with default parameters

kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(text)

for kw, score in keywords:
    print(f"{kw} ({score})")

With custom parameters

custom_kw_extractor = yake.KeywordExtractor(
    lan="en",              # language
    n=3,                   # ngram size
    dedupLim=0.9,          # deduplication threshold
    dedupFunc='seqm',      # deduplication function
    windowsSize=1,         # context window
    top=10,                # number of keywords to extract
    features=None          # custom features
)

keywords = custom_kw_extractor.extract_keywords(text)

In [18]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50) #keeping 50 for now, but it could be more concise to do less per document 
kw_extractor

In [ ]:
# Lister les fichiers, advantage is that we can see which files mention which keywords but we can also do the same with a bash eventually
data_path = "../../data/halley/halley_txt/"
files = [f for f in os.listdir(data_path) if f.endswith('.txt')]
len(files)

551

In [24]:
# Tokenize each file individually
#I think it's best to first see all of they keywords per file before aggregating and then doing one big word cloud for the whole set, or just a certain period. I would've wanted to compare the wordcloud of the different periods ngl. I mean, at least I could do that for la libre belgique or whatever it was that was also reporting in 1835

data_path = "../../data/halley/halley_txt/"
tokenised_files = [f for f in os.listdir(data_path) if f.endswith('.txt')]

tokens_by_file = {}

for filename in tokenised_files:
    filepath = os.path.join(data_path, filename)
    
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read() # Récupérer le texte du fichier sans le jointure 
    
    # Tokenize
    tokens = nltk.wordpunct_tokenize(text) # Extraire les tokens du texte en cours
    
    # Filtrer
    filtered_tokens = [w.lower() for w in tokens 
                      if len(w) > 2 # Ne garder que les bigrammes
                      and w.isalpha() 
                      and w.lower() not in sw
                      and not re.search(r'(.)\1{2,}', w.lower())]
    
    tokens_by_file[filename] = filtered_tokens

print(f"Tokenized {len(tokens_by_file)} files")
print(f"Total tokens across all files: {sum(len(tokens) for tokens in tokens_by_file.values())}")
print(len(tokenised_files)) #just for me to be sure that indeed they were all read

# Show example for first file
first_file = tokenised_files[0] # Limiter à 1 fichier pour l'exemple, need to do a bash of all? after checking stop words?
print(f"\n{first_file}: {len(tokens_by_file[first_file])} tokens")
print(f"First 20 tokens: {tokens_by_file[first_file][:20]}") #helllll yeah she's alive and didn't take 15 years to run 

Tokenized 551 files
Total tokens across all files: 1584196
551

KB_JB1051_1927-08-25_01-00004.txt: 1677 tokens
First 20 tokens: ['rouge', 'edison', 'revendique', 'priorité', 'invention', 'phonographe', 'signalé', 'dernièrement', 'mort', 'poète', 'savant', 'français', 'charles', 'croa', 'considéré', 'inventeur', 'phonographe', 'décrit', 'mécanisme', 'edison', 'appliquer', 'développer', 'invention', 'savant', 'fronçais', 'interrogé', 'point', 'edison', 'vient', 'donne', 'précisions', 'juillet', 'conçus', 'idée', 'pliu', 'recherche', 'pétroles', 'synthétiques', 'thomas', 'edison', 'grand', 'inventeur', 'américain', 'vient', 'fêter', 'anniversaire', 'nographe', 'août', 'appareil', 'construit', 'côté']


In [ ]:
#start here
for f in sorted(tokenised_files)[:10]:
    tokenised_text = open(os.path.join(data_path, f), 'r', encoding="utf-8").read()
    token_keywords = kw_extractor.extract_keywords(tokenised_text)
    kept_tokens = []
    for kw, score in token_keywords:
        token_words = kw.split()
        if len(token_words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept_tokens )}...")

#returns nothing :( but runs through the first 10 files :) 

KB_JB1051_1927-08-25_01-00004.txt mentions these keywords: ...
KB_JB1051_1937-11-14_01-00006.txt mentions these keywords: ...
KB_JB1051_1939-08-06_01-00005.txt mentions these keywords: ...
KB_JB1051_1957-04-18_01-00004.txt mentions these keywords: ...
KB_JB1051_1957-10-14_01-00001.txt mentions these keywords: ...
KB_JB421_1902-10-19_01-00001.txt mentions these keywords: ...
KB_JB421_1907-10-26_01-00002.txt mentions these keywords: ...
KB_JB421_1909-05-07_01-00003.txt mentions these keywords: ...
KB_JB421_1909-08-06_01-00001.txt mentions these keywords: ...
KB_JB421_1909-10-23_01-00001.txt mentions these keywords: ...


# FROM HERE DOWN WE'RE IN DRAFT MODE 
Tokenised all the files, now I'm working on kw extraction for each of the files so we can start comparing, maybe then only take the lowest vectors from the dataset and start making the word cloud from there (that way the keywords of EACH file are preserved, not just the bashed kw (actually wait it'd be interesting to see if they'd be different(I should include that in the analysis as the limits of NLP???)))

In [ ]:
# Extraire les mots clés du text bash 

keywords = kw_extractor.extract_keywords(text) #I think text here is the whole halley text
#doesn't look like stopwords were removed here

keywords 

[('ligne', 0.006576956516920981),
 ('grande ligne', 0.010414814932783081),
 ('jjjjgjs et Jugements', 0.012405281434844163),
 ("d'un", 0.012768895090072173),
 ("qu'il", 0.01404661186692881),
 ('rue', 0.018646918396135623),
 ('petite ligne', 0.022057795950928976),
 ('Belgique', 0.022894960513796105),
 ('Faits-drt ANNONCES', 0.02459954981037721),
 ('service', 0.02593999816391375),
 ("qu'ils", 0.032775427689500564),
 ('bons', 0.03291445281262834),
 ('faire', 0.03583407481960633),
 ('bons vieux serviteurs', 0.03631056971969405),
 ('TRAITE A FORFAIT', 0.03783214099594524),
 ('bon', 0.0384001949480664),
 ('prince', 0.039173282293782995),
 ('Albert', 0.041280351933826676),
 ('prince Albert', 0.04207603403127464),
 ("c'est", 0.042311261418854614),
 ("d'une", 0.044920043564026534),
 ('rue Clément XIV', 0.04628643594921162),
 ("qu'il possède rue", 0.05113952925236211),
 ("l'on", 0.053223547434741274),
 ('VIEUX', 0.054648719572990195),
 ("service d'un", 0.05596176939282363),
 ("service d'un réel",

Keywords Analysis

In [ ]:
# Choisir un fichier => actually I think I want this to do all of the files, if this is the analysis I'm thinking of but let's test as is for now
#maybe I could do just the texts during may 1910? I could filter the files based on date either using the setup from preparer_halley or just split the names again

this_file = files[0]
this_file

# Récupérer le texte du fichier
text = open(os.path.join(data_path, this_file), 'r', encoding='utf-8').read()
text[:500] #gets first 500 characters, Just to check the beginning of the text, I can change this later

"yF: LE DRAPEAU ROUGE Edison revendique la priorité de l'invention du phonographe .- On a signalé dernièrement la mort du poète ot'savant français Charles Croa considéré com- , mo l'inventeur du phonographe, dont il aurait décrit, le premier, le mécanisme. Edison n'aurait l'ait qu'appliquer et développer l'invention du savant fronçais. Interrogé sur ce point, Edison vient de donne! les précisions suivantes : « — Jo no puis vous dire que ceci : c'est en ■ juillet 1977 que je conçus l'idée de mon p"

### Extract Keywords from All Files with Time Filtering

In [62]:
def extract_keywords_from_halley_texts(specific_years=None, specific_months=None, ngram_size=1):
    """
    Extract keywords from Halley texts with optional time filtering.

    Based on the structure from preparer_halley.ipynb for filename parsing
    and filtering by time period. Processes files individually (not halley_all.txt)
    to maintain document boundaries for YAKE and enable time-based filtering.

    Parameters:
    - specific_years: list of year strings, e.g., ['1910', '1911']
    - specific_months: list of (year, month) tuples, e.g., [('1910', '05'), ('1910', '06')]
    - ngram_size: int, controls n-gram filtering (1=unigrams, 2=bigrams, etc.)
                  Change this parameter to easily switch between unigrams/bigrams/trigrams

    Returns:
    - Dictionary: {filename: [(keyword, score), ...]}
    """
    keywords_by_file = {}

    # Iterate through all text files in the Halley directory
    for f in sorted(files):
        if "_" in f and f.endswith("txt"):
            # Parse filename to extract date metadata (same logic as preparer_halley.ipynb)
            elems = f.split("_")
            year = elems[2].split("-")[0]
            month = elems[2].split("-")[1]

            # Apply time filters if specified
            if specific_years and year not in specific_years:
                continue
            if specific_months and (year, month) not in specific_months:
                continue

            # Read the text file
            text = open(os.path.join(data_path, f), 'r', encoding='utf-8').read()

            # Extract keywords using YAKE
            keywords = kw_extractor.extract_keywords(text)

            # Filter keywords by n-gram size (number of words in the keyword)
            # To change from unigrams to bigrams/trigrams, just modify ngram_size parameter
            kept = [(kw, score) for kw, score in keywords if len(kw.split()) == ngram_size]

            # Store in dictionary organized by filename
            keywords_by_file[f] = kept

    return keywords_by_file

In [65]:
# Extract unigram keywords from all Halley texts, organized by file
# To extract bigrams instead, change ngram_size=2
keywords_by_file = extract_keywords_from_halley_texts(ngram_size=1)
print(f"Extracted keywords from {len(keywords_by_file)} files")

Extracted keywords from 551 files


In [66]:
# Flatten keywords from all files (keeping duplicates for frequency counting)
all_keywords_flat = [kw for kws in keywords_by_file.values() for kw, score in kws]

# Calculate frequency distribution
keyword_freq = Counter(all_keywords_flat)

# Display results
print(f"Total keyword occurrences: {len(all_keywords_flat)}")
print(f"Unique keywords: {len(keyword_freq)}")
print(f"\nTop 20 most frequent keywords:")
keyword_freq.most_common(20)

Total keyword occurrences: 19655
Unique keywords: 4842

Top 20 most frequent keywords:


[("qu'il", 274),
 ("d'une", 265),
 ("d'un", 261),
 ('heures', 242),
 ('Bruxelles', 228),
 ('faire', 225),
 ('rue', 185),
 ('ans', 185),
 ('fut', 184),
 ('jour', 170),
 ('grand', 156),
 ('Paris', 153),
 ("C'est", 147),
 ("c'est", 145),
 ('Belgique', 138),
 ('francs', 138),
 ('temps', 132),
 ('jours', 132),
 ('lieu', 125),
 ('dit', 118)]

In [ ]:
# Example 1: Keywords from 1910 only
keywords_1910 = extract_keywords_from_halley_texts(specific_years=['1910'], ngram_size=1)
print(f"Files from 1910: {len(keywords_1910)}")

# Example 2: Keywords from May 1910 (peak Halley's passage)
may_1910_keywords = extract_keywords_from_halley_texts(specific_months=[('1910', '05')], ngram_size=1)
print(f"Files from May 1910: {len(may_1910_keywords)}")

# Example 3: Compare May vs. all of 1910
freq_may = Counter([kw for kws in may_1910_keywords.values() for kw, score in kws])
print(f"\nMay 1910 top keywords: {freq_may.most_common(10)}")

#ok need to figure out why the stopwords lists aren't working here

Files from 1910: 242
Files from May 1910: 98

May 1910 top keywords: [('heures', 61), ("d'un", 60), ('Bruxelles', 58), ("d'une", 58), ("qu'il", 55), ('jour', 43), ('faire', 41), ("C'est", 36), ('rue', 35), ('ans', 34)]


In [ ]:
# Show first 3 files and their keywords to verify results
for i, (filename, keywords) in enumerate(list(keywords_by_file.items())[:3]):
    print(f"\n{filename}:")
    print(f"  Top 10 keywords: {keywords[:10]}")